# Tutorial: Variational algorithms on Hybrid jobs

## Prepare files

### Prepare Hybrid job script

In [1]:
! cat ./data/2_hybrid_jobs/job_script.py

"""Example of Hybrid Job payload with VQE."""
from braket.jobs import save_job_result
from qiskit.quantum_info import SparsePauliOp
from qiskit.algorithms.minimum_eigensolvers import VQE
from qiskit.algorithms.optimizers import SLSQP
from qiskit.circuit.library import TwoLocal
from qiskit.primitives import BackendEstimator

from qiskit_braket_provider import AWSBraketProvider


def main():
    backend = AWSBraketProvider().get_backend("SV1")

    h2_op = SparsePauliOp(
        ["II", "IZ", "ZI", "ZZ", "XX"],
        coeffs=[
            -1.052373245772859,
            0.39793742484318045,
            -0.39793742484318045,
            -0.01128010425623538,
            0.18093119978423156,
        ],
    )

    estimator = BackendEstimator(
        backend=backend,
        options={"seed_simulator": 42, "seed_transpiler": 42, "shots": 10},
        skip_transpilation=False,
    )
    ansatz = TwoLocal(rotation_blocks="ry", entanglement_blocks="cz")
    slsqp = SLSQP(maxiter=1)

    vqe = 

### Prepare docker image

In [3]:
! cat ./data/2_hybrid_jobs/Dockerfile

FROM 292282985366.dkr.ecr.us-west-2.amazonaws.com/amazon-braket-base-jobs:1.0-cpu-py37-ubuntu18.04

RUN python3 -m pip install --upgrade pip

RUN python3 -m pip install --no-cache --upgrade git+https://github.com/qiskit-community/qiskit-braket-provider


## Prepare env

### Login to AWS CLI and docker

Login to AWS' ERC to pull base image

`aws ecr get-login-password --region us-west-2 | docker login --username AWS --password-stdin 292282985366.dkr.ecr.us-west-2.amazonaws.com`

Login to yours ERC to push your image

`aws ecr get-login-password --region <YOUR_AWS_REGION> | docker login --username AWS --password-stdin <YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_AWS_REGION>.amazonaws.com`

### Create ECR repo

`! aws ecr create-repository --repository-name amazon-braket-qiskit-provider`

### Build docker image

`! docker build -t amazon-braket-qiskit-provider ./data/2_hybrid_jobs/`

`! docker tag amazon-braket-qiskit-provider:latest <YOUR_ACCOUNT_ID>.dkr.ecr.<YOUR_AWS_REGION>.amazonaws.com/amazon-braket-qiskit-provider:latest`

## Running your job

In [44]:
from pprint import pprint

from braket.aws import AwsQuantumJob

image_uri = "<ACCOUNT_ID>.dkr.ecr.<AWS_REGION>.amazonaws.com/amazon-braket-qiskit-provider:latest"

job = AwsQuantumJob.create(
    device="arn:aws:braket:::device/quantum-simulator/amazon/sv1",
    source_module="data/2_hybrid_jobs/job_script.py",
    entry_point="job_script:main",
    wait_until_complete=False,
    job_name="qiskit-braket-vqe-6",
    image_uri=image_uri,
)

In [45]:
pprint(job.result())

{'VQE': {'eigenstate': {'01': 0.8366600265340756, '11': 0.5477225575051661},
         'eigenvalue': -1.6823050629682066,
         'optimal_parameters': [2.965910707685307,
                                -4.658133055892289,
                                -4.3956015975227265,
                                -3.682189424847994,
                                0.3499389384822065,
                                -2.55160387237748,
                                2.0897918250881915,
                                1.0061448749802206],
         'optimal_point': [2.965910707685307,
                           -4.658133055892289,
                           -4.3956015975227265,
                           -3.682189424847994,
                           0.3499389384822065,
                           -2.55160387237748,
                           2.0897918250881915,
                           1.0061448749802206],
         'optimal_value': -1.6823050629682066}}
